## Presentation Type Tools Explained






In [21]:
import intervals
from intervals import * 
from intervals import main_objs
import pandas as pd
import re
import os
import numpy
import itertools
# import collections
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")
    
MUSDIR = ("Music_Files")
CHECK_FOLDER = os.path.isdir(MUSDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MUSDIR)
    print("created folder : ", MUSDIR)

else:
    print(MUSDIR, "folder already exists.")

saved_csv folder already exists.
Music_Files folder already exists.


In [22]:
# Select a prefix:

# prefix = 'Music_Files/'
# just add the CRIM Piece ID here
prefix = 'https://crimproject.org/mei/'
# just add the CRIM Piece ID here
mei_file = 'CRIM_Model_0008.mei'
url = prefix + mei_file
piece = importScore(url)

print(piece.metadata)

Previously imported piece detected.
{'title': 'Ave Maria', 'composer': 'Josquin Des Prés'}


# How does it Work?

### Find the Melodic Entries

With various options, then passed to the entries method:

```
nr = piece.notes(combineUnisons=True)
mel = piece.melodic(df=nr, kind='d', end=False)
mel_ng = piece.ngrams(df=mel, n=4)
entries = piece.entries(mel_ng)
ng_durs = piece.durations(df=entries)
det = piece.detailIndex(nr, offset=True, progress=True)
entries.head(15)
```


OR simply with defaults:

```
piece.entries(df=None, n=None, thematic=False, anywhere=False, fermatas=True, exclude=[])
```

In [48]:

nr = piece.notes(combineUnisons=True)
mel = piece.melodic(df=nr, kind='d', end=False)
mel_ng = piece.ngrams(df=mel, n=4)
entries = piece.entries(mel_ng)
ng_durs = piece.durations(df=entries)
det = piece.detailIndex(nr, offset=True, progress=True)
entries.head(15)


# piece.entries()

,[Superius],Altus,Tenor,Bassus
0.0,"(4, 2, 2, -3)",NaN,NaN,NaN
16.0,NaN,"(4, 2, 2, -3)",NaN,NaN
32.0,NaN,NaN,"(4, 2, 2, -3)",NaN
48.0,NaN,NaN,NaN,"(4, 2, 2, -3)"
56.0,"(-2, -2, -2, 2)",NaN,NaN,NaN
72.0,NaN,"(-2, -2, -2, 2)",NaN,NaN
88.0,NaN,NaN,"(-2, -2, -2, 2)",NaN
104.0,NaN,NaN,NaN,"(-2, -2, -2, 2)"
124.0,"(2, 2, -3, -2)",NaN,NaN,NaN
156.0,NaN,NaN,"(2, 2, -3, 3)",NaN


In [50]:
points = pd.DataFrame(columns=['Composer',
                    'Title',
                    'First_Offset',
                    'Measures_Beats',
                    'Melodic_Entry_Intervals',
                    'Offsets',
                    'Soggetti',
                    'Time_Entry_Intervals',
                    'Voices',
                    'Presentation_Type'])
points2 = pd.DataFrame()
# defines column order in final df
# others are at the end for the overlapping entries
col_order = list(points.columns) + ['Number_Entries',
                                    'Flexed_Entries',
                                    'Parallel_Entries',
                                    'Parallel_Voice',
                                    'Count_Offsets',
                                    'Offsets_Key']

det = piece.detailIndex(nr, offset=True, progress=True)
points

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type


### Stack the Entries

- stack the entries as one column--in the order of their appearance
- We also rename a column to clarify that this is our 'pattern'

In [26]:

mels_stacked = entries.stack().to_frame()
mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)
mels_stacked

pattern
0.0    [Superius]    (4, 2, 2, -3)
16.0   Altus         (4, 2, 2, -3)
32.0   Tenor         (4, 2, 2, -3)
48.0   Bassus        (4, 2, 2, -3)
56.0   [Superius]  (-2, -2, -2, 2)
...                            ...
1200.0 Tenor        (-2, -2, 2, 2)
       Bassus      (5, -2, -2, -2)
1236.0 [Superius]  (-2, -2, -2, 2)
       Tenor        (-2, 2, 2, -2)
       Bassus       (8, -4, 2, -2)

[74 rows x 1 columns]

### Edit Distances for Similar Entries

- find the edit distance, based on side-by-side comparison of melodic ngrams
- note that we can provide a 'head_flex' that allows small differences in the first interval to pass the test


In [27]:
# set the head flex:
head_flex = 1

# now use that flex in the "flexed_distance" function
dist = piece.flexed_distance(head_flex, entries)

# stack the result
dist.stack().to_frame()


0
(4, 2, 2, -3)  (4, 2, 2, -3)     0
               (-2, -2, -2, 2)  19
               (2, 2, -3, -2)    8
               (2, 2, -3, 3)    13
               (-3, -2, 2, 2)   16
...                             ..
(8, -4, 2, -2) (-2, -2, 2, -2)  12
               (-3, 4, -2, -2)  23
               (-2, -2, 2, 2)   16
               (-2, 2, 2, -2)   16
               (8, -4, 2, -2)    0

[1089 rows x 1 columns]

### Body Flex

- filter body flex distances to threshold.  <2 is good

In [18]:
# set body flex
body_distance = 1

# filter the previous dataframe (which already considers the head flex)
filtered_dist_stack = dist_stack[dist_stack[0] < body_distance]

# restack these
filtered_dist = filtered_dist_stack.reset_index()
filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)
filtered_dist.head()


,source,match,0
0,"(4, 2, 2, -3)","(4, 2, 2, -3)",0
1,"(-2, -2, -2, 2)","(-2, -2, -2, 2)",0
2,"(2, 2, -3, -2)","(2, 2, -3, -2)",0
3,"(2, 2, -3, 3)","(2, 2, -3, 3)",0
4,"(-3, -2, 2, 2)","(-3, -2, 2, 2)",0


### Grouping Filtered Matches

- Group the filtered distanced patterns

In [28]:

full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()
full_list_of_matches.head()

,source,match
0,"(-2, -2, -2, -2)","[(-2, -2, -2, -2)]"
1,"(-2, -2, -2, 2)","[(-2, -2, -2, 2)]"
2,"(-2, -2, -3, 5)","[(-2, -2, -3, 5)]"
3,"(-2, -2, 2, -2)","[(-2, -2, 2, -2)]"
4,"(-2, -2, 2, -3)","[(-2, -2, 2, -3)]"


### With the Matches Identified, Find the Locations and Distances Between Them

- Using the "matches" from the previous step, return a series of arrays:  all of the instances of a given soggetto, with offset and voice

In [55]:
# just decide in advance that we are not looking for hidden types.
include_hidden_types = False

# Group the filtered distanced patterns
full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()


# classification without hidden types
if include_hidden_types == False:
    for matches in full_list_of_matches["match"]:
        
        # build a list of the related entries--
        # those that qualify as matching according to initial settings for flex head and body
        related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
        entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
        offset_list = entry_array.index.to_list()
        
        # split them when we encounter a long gap (> 70 offsets)
        split_list = list(ImportedPiece._split_by_threshold(offset_list))
        
        # classify each 'set'
        # note that the melodic entry distances are determined by the helper: _temp_dict_of_details
        # the data are temporarily stored in "temp", then appended to "points" in the next steps
        for item in split_list:
            temp = piece._temp_dict_of_details(item, entry_array, det, matches)
            points = points.append(temp, ignore_index=True)
            points['Presentation_Type'] = points['Time_Entry_Intervals'].apply(ImportedPiece._classify_by_offset)
            points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
            points = points[points['Offsets'].apply(len) > 1]

    
    # cleaning up and recording various other features:
    # offsets
    points["Offsets_Key"] = points["Offsets"].apply(ImportedPiece._offset_joiner)
    
    # flexed entries ?
    points['Flexed_Entries'] = points["Soggetti"].apply(len) > 1
    
    # how many entries ?
    points["Number_Entries"] = points["Offsets"].apply(len)
    
    # makes sure we have no p types with only one entry
    points["Count_Offsets"] = points["Offsets"].apply(set).apply(len)
    points = points[points["Count_Offsets"] > 1]

    # sorts results by first offset
    points = points.reindex(columns=col_order).sort_values("First_Offset").reset_index(drop=True)
    
    # applying various private functions for overlapping entry tests.
    # note that ng_durs must be passed to the first of these, via args
    
    points["Entry_Durs"] = points[["Offsets", "Voices"]].apply(ImportedPiece._dur_ngram_helper, args=(ng_durs,), axis=1)
    points["Overlaps"] = points[["Entry_Durs", "Offsets"]].apply(ImportedPiece._entry_overlap_helper, axis=1)
    points["Count_Non_Overlaps"] = points["Overlaps"].apply(ImportedPiece._non_overlap_count)
    
    # remove columns no longer needed
    points.drop(['Count_Offsets', 'Offsets_Key', 'Entry_Durs', 'Overlaps'], axis=1, inplace=True)

    
points

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries,Parallel_Entries,Parallel_Voice,Count_Non_Overlaps
0,Josquin Des Prés,Ave Maria,0.0,"[1/1.0, 3/1.0, 5/1.0, 7/1.0]","[P-8, P1, P-8]","[0.0, 16.0, 32.0, 48.0]","[(4, 2, 2, -3)]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]",PEN,4,False,0.0,None,0
1,Josquin Des Prés,Ave Maria,56.0,"[8/1.0, 10/1.0, 12/1.0, 14/1.0]","[P-8, P1, P-8]","[56.0, 72.0, 88.0, 104.0]","[(-2, -2, -2, 2)]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]",PEN,4,False,0.0,None,0
2,Josquin Des Prés,Ave Maria,176.0,"[23/1.0, 28/1.0]",[P-8],"[176.0, 216.0]","[(-3, -2, 2, 2), (-2, -2, 2, 2)]",[40.0],"[Altus, Bassus]",FUGA,2,True,0.0,None,0
3,Josquin Des Prés,Ave Maria,244.0,"[31/3.0, 35/3.0]",[P-8],"[244.0, 276.0]","[(2, 2, 2, -2)]",[32.0],"[[Superius], Tenor]",FUGA,2,False,0.0,None,0
4,Josquin Des Prés,Ave Maria,428.0,"[54/3.0, 55/1.0, 59/3.0, 60/1.0]","[P-5, P-4, P-5]","[428.0, 432.0, 468.0, 472.0]","[(4, -2, 2, 2)]","[4.0, 36.0, 4.0]","[[Superius], Altus, Tenor, Bassus]",ID,4,False,0.0,None,0
5,Josquin Des Prés,Ave Maria,508.0,"[64/3.0, 66/3.0, 67/3.0, 69/3.0]","[P-5, P-4, P-5]","[508.0, 524.0, 532.0, 548.0]","[(2, -3, 2, -3)]","[16.0, 8.0, 16.0]","[[Superius], Altus, Tenor, Bassus]",ID,4,False,0.0,None,0
6,Josquin Des Prés,Ave Maria,616.0,"[78/1.0, 81/1.0]",[P-8],"[616.0, 640.0]","[(2, -3, 2, -2)]",[24.0],"[[Superius], Tenor]",FUGA,2,False,0.0,None,0
7,Josquin Des Prés,Ave Maria,748.0,"[94/1.0, 98/1.0]",[P1],"[748.0, 796.0]","[(2, -2, -2, -2)]",[48.0],"[Altus, Altus]",FUGA,2,False,0.0,None,1
8,Josquin Des Prés,Ave Maria,796.0,"[98/1.0, 98/2.0]",[P-5],"[796.0, 800.0]","[(2, 2, -3, 4)]",[4.0],"[[Superius], Tenor]",FUGA,2,False,0.0,None,0
9,Josquin Des Prés,Ave Maria,944.0,"[111/1.0, 114/1.0, 119/1.0, 122/1.0]","[P-8, P8, P-8]","[944.0, 968.0, 1008.0, 1032.0]","[(2, -2, -2, -2)]","[24.0, 40.0, 24.0]","[[Superius], Tenor, [Superius], Tenor]",ID,4,False,0.0,None,0


### Split the Lists so We Avoid Long Gaps

In [82]:
# gets Progress!

temp = p_types.set_index("First_Offset")
temp2 = temp
piece.di(temp2, progress=True, beat=False, measure=False)

,Composer,Title,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries,Parallel_Entries,Parallel_Voice,Count_Non_Overlaps
Progress,,,,,,,,,,,,,,
0.000000,Johannes Lupi,Veni speciosam,"[1/1.0, 5/1.0, 9/2.0, 10/1.0]","[P-8, P1, P8]","[0.0, 32.0, 66.0, 72.0]","[(5, -2, 2, 3, -2)]","[32.0, 34.0, 6.0]","[Superius, PrimusTenor, SecundusTenor, Superius]",FUGA,4,False,0.0,None,0
0.011194,Johannes Lupi,Veni speciosam,"[2/3.0, 6/3.0]",[P-8],"[12.0, 44.0]","[(4, -2, 2, 3, -2)]",[32.0],"[Contratenor, Bassus]",FUGA,2,False,0.0,None,0
0.087687,Johannes Lupi,Veni speciosam,"[12/4.0, 16/4.0]",[P4],"[94.0, 126.0]","[(-3, 2, 2, -2, -2)]",[32.0],"[Bassus, Bassus]",FUGA,2,False,0.0,None,1
0.128731,Johannes Lupi,Veni speciosam,"[18/2.0, 18/4.0, 20/4.0, 21/2.0, 24/2.0, 24/4....","[P-4, P8, P5, M-9, P-4, P4]","[138.0, 142.0, 158.0, 162.0, 186.0, 190.0, 198.0]","[(-2, -3, 2, 2, 2)]","[4.0, 16.0, 4.0, 24.0, 4.0, 8.0]","[PrimusTenor, Bassus, Contratenor, Superius, P...",FUGA,7,False,0.0,None,0
0.210821,Johannes Lupi,Veni speciosam,"[29/2.0, 31/2.0, 34/4.0]","[P5, P1]","[226.0, 242.0, 270.0]","[(5, -2, 2, 3, -2)]","[16.0, 28.0]","[PrimusTenor, Superius, Contratenor]",FUGA,3,False,0.0,None,0
0.220149,Johannes Lupi,Veni speciosam,"[30/3.0, 31/3.0, 35/2.0, 37/2.0]","[P5, P8, P-8]","[236.0, 244.0, 274.0, 290.0]","[(4, -2, 2, 3, -2)]","[8.0, 30.0, 16.0]","[Bassus, SecundusTenor, Superius, PrimusTenor]",FUGA,4,False,0.0,None,0
0.292910,Johannes Lupi,Veni speciosam,"[40/2.0, 41/2.0, 43/2.0, 43/4.0, 45/2.0]","[P8, P-12, m9, M7]","[314.0, 322.0, 338.0, 342.0, 354.0]","[(2, -3, 2, 3, -2)]","[8.0, 16.0, 4.0, 12.0]","[Bassus, Superius, Bassus, Contratenor, Superius]",FUGA,5,False,0.0,None,0
0.302239,Johannes Lupi,Veni speciosam,"[41/3.0, 42/2.0, 44/4.0, 46/2.0]","[P-4, M-2, P5]","[324.0, 330.0, 350.0, 362.0]","[(2, -3, 2, 4, -2)]","[6.0, 20.0, 12.0]","[PrimusTenor, SecundusTenor, PrimusTenor, Secu...",FUGA,4,False,0.0,None,0
0.401119,Johannes Lupi,Veni speciosam,"[54/4.0, 55/2.0, 56/2.0, 58/4.0, 60/4.0, 62/2....","[P4, P5, P-5, P-4, P-5, P4, P1]","[430.0, 434.0, 442.0, 462.0, 478.0, 490.0, 492...","[(4, 1, -2, 2, -2)]","[4.0, 8.0, 20.0, 16.0, 12.0, 2.0, 26.0]","[PrimusTenor, Contratenor, Superius, PrimusTen...",FUGA,8,False,0.0,None,1
